In [17]:
import pandas as pd
import numpy as np

from pathlib import Path
import csv

import os
import requests
import json

import quandl

from dotenv import load_dotenv
load_dotenv()

True

In [18]:
## For TI data 
from datetime import date
from scipy import stats
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad

In [19]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [20]:
## Get Short "Interest" Data from Quandl 
## QUANDL/NASDAQ 
nsdq_api_key = os.environ.get('NASDAQ_API_KEY')
base_url_nsdq = 'https://data.nasdaq.com/api/v3/datasets/FINRA/'

def get_short_data_QUANDL(symbol):
    string_nsdq = "FINRA/FNSQ_"+symbol
    string_nyse = "FINRA/FNYX_"+symbol
    
    df1 = quandl.get(string_nsdq,start_date=start_date,end_date=end_date,authtoken=nsdq_api_key)   ## Nasdaq
    df2 = quandl.get(string_nyse,start_date=start_date,end_date=end_date,authtoken=nsdq_api_key)   ## NYSE

    df1 = df1.rename(columns={'ShortVolume':'ShortVolumeNSDQ','TotalVolume':'TotalVolumeNSDQ'})
    #df1 = df1.drop(columns={'ShortExemptVolume'})
    df1 = df1.rename(columns={'ShortExemptVolume':'ShortExemptVolumeNSDQ'})

    df2 = df2.rename(columns={'ShortVolume':'ShortVolumeNYSE','TotalVolume':'TotalVolumeNYSE'})
    #df2 = df2.drop(columns={'ShortExemptVolume'})
    df2 = df2.rename(columns={'ShortExemptVolume':'ShortExemptVolumeNYSE'})

    df3 = pd.merge(df1,df2,on='Date',how='outer')
    #df3 = df3.fillna(0)
    
    return df3


## Not needed 

# ## Return FTD Data from SEC FTD files using a Stock's CUSIP number to sort 
# def return_ftd_data_cusip(cusip_number):
#     df = ftd_df.copy()
#     df.set_index("CUSIP",inplace=True)
#     df = df.loc[cusip_number]
#     df['Date'] = pd.to_datetime(df['Date'])
#     df = df.set_index('Date')
#     return df

# ## Return the CUSIP symbol from the symbol_df symbol list 
# def return_CUSIP_from_symbol(symbol):
#     df = symbol_df.copy()
#     df.set_index('SYMBOL',inplace=True)
#     cusip_variable = df.loc[symbol]
#     cusip_variable = cusip_variable['CUSIP']
#     return cusip_variable

# def return_ftd_data_symbol(symbol):
#     cusip_number = return_CUSIP_from_symbol(symbol)
#     df = return_ftd_data_cusip(cusip_number)
#     return df

In [21]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

## FMP Functions 
def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
    url_hist_price = fmpbase_urlv3+'historical-price-full/'
    url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
    resp_data = requests.get(url_hist_query_with_date)
    json_ = resp_data.json()
    data = json_['historical']
    df = pd.DataFrame(data)
    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
    df.set_index('Date',inplace=True)
    df.drop(columns='label',inplace=True)
    return df

# def get_float_data_FMP(symbol):
#     url_float_shares = fmpbase_urlv4+'shares_float?symbol='
#     url_query_float_data = url_float_shares+symbol+'&apikey='+api_key
#     resp_data = requests.get(url_query_float_data)
#     #df = pd.DataFrame(resp_data.json())
#     json_ = resp_data.json()
#     return json_[0]

# def get_company_profile_FMP_json(symbol):
#     ## https://fmpcloud.io/api/v3/profile/AAPL?apikey='yourkeyhere'
#     url_company_profile_url = fmpbase_urlv3+'profile/'+symbol+'?apikey='+api_key
#     resp_data = requests.get(url_company_profile_url)
#     json_response = resp_data.json()
#     return json_response[0]

# # def save_and_export_raw_df_csv(data, symbol, path='None'):
# #     ## Can set custom path (useful for testing) otherwise will default to below path
# #     if path=='None':
# #         path = ('../FilesExportIndividualStockDFs_Big/'+symbol+'_combined_df.csv')
# #     data.to_csv(path)
    
# def save_and_export_raw_df_pkl(data, symbol, path='None'):
#     ## Can set custom path (useful for testing) otherwise will default to below path
#     if path=='None':
#         path = ('../FilesExport_TimeSeries_DFs/'+symbol+'_combined_df.pkl')
#     save_obj(data,path)

In [22]:
def update_data(symbol):
    
    ## Get updated data
    df = get_FMP_historical_data(symbol)    
    df2 = get_short_data_QUANDL(symbol)
    df = pd.merge(df,df2,on='Date',how='outer')
    df = df.fillna(0)
    
    ## Import old dataframe, drop FTDs, join data and save 
    path = Path('../FilesExport_Complete_DFs_TI_noShift/'+symbol+'_TI_DF_no_shift.pkl')
    data = load_obj(path)
    import_df = data[symbol].drop(columns='QUANTITY_FAILS')
    
    updated_df = import_df.append(df)
    
    return updated_df    

In [31]:
## Setup TI functions 

def get_dataframe(symbol):

    #data = pd.read_csv("../FilesExportIndividualStockDFs_Big/"+ticker+"_combined_df.csv", index_col='Date', parse_dates=True)
    # path = Path('../FilesExport_Complete_DataDicts/'+symbol+'_data_dict.pkl')
    # data_import = load_obj(path)
    # data = data_import[symbol]['dataFrame']

    ## Use newly made function to get_dataframe 
    
    data = update_data(symbol)
    
    return data



def rolling_spearman(seqa, seqb, window):
    stridea = seqa.values.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.values.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    ar = pd.DataFrame(ssa)
    br = pd.DataFrame(ssb)
    ar = ar.rank(1)
    br = br.rank(1)
    corrs = ar.corrwith(br, 1)
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)

def bollinger_bands(dataframe,period=20):
    data = dataframe.copy()
    data['middle_band'] = data[['adjClose']].rolling(window=period).mean()
    data[str(period)+'_day_stdev'] = data[['adjClose']].rolling(window=period).std()
    data['upper_band'] = data['middle_band']+2*data[str(period)+'_day_stdev']
    data['lower_band'] = data['middle_band'] - 2*data[str(period)+'_day_stdev']
    data['spread'] = data['upper_band'] + data['lower_band']
    data['change_in_spread'] = data['spread']/data['spread'].shift(1)-1
    data[str(period)+"_return"] = data['adjClose']/data['adjClose'].shift(period)-1
    data['bollinger_signal'] = data['change_in_spread'].rank(ascending=False, pct=True)
    data.dropna()

    return data

def dema(dataframe, period1=10, period2=20):
    data = dataframe.copy()
    data[str(period1)+'ema1'] = dataframe[['adjClose']].ewm(span=period1, adjust=False).mean()
    data[str(period1)+'ema2'] = data[str(period1)+'ema1'].ewm(span=period1, adjust=False).mean()
    data[str(period1)+'dema'] = 2*data[str(period1)+'ema1'] - data[str(period1)+'ema2']
    data[str(period2)+'ema1'] = data[['adjClose']].ewm(span=period2, adjust=False).mean()
    data[str(period2)+'ema2'] = data[str(period2)+'ema1'].ewm(span=period2, adjust=False).mean()
    data[str(period2)+'dema'] = 2*data[str(period2)+'ema1'] - data[str(period2)+'ema2']
    data[str(period1)+"_return"] = data['adjClose']/data['adjClose'].shift(period1)-1
    data['spread'] = data[str(period1)+'dema'] - data[str(period2)+'dema']
    data['dema_signal'] = data['spread'].rank(ascending=True, pct=True)
    #data = data.dropna()
    return data

def price_momentum(dataframe, smoothing1=0.0571, smoothing2=0.1, periods1=15, periods2=10):
    data = dataframe.copy()
    data['smoothing_factor'] = smoothing1
    data[str(periods1)+"average"] = data['changeOverTime'].rolling(window=periods1).mean()
    smoothing_factor_list = [data.iloc[periods1][str(periods1)+"average"]]
    data = data.dropna()
    i=1
    j=0
    while i < len(data[str(periods1)+"average"]):
        smoothing_factor = data.iloc[i]['changeOverTime']*data.iloc[i]['smoothing_factor'] + smoothing_factor_list[j]*(1-data.iloc[i]['smoothing_factor'])
        smoothing_factor_list.append(smoothing_factor)
        j+=1
        i+=1
    data['35d_custom_smoothing'] = smoothing_factor_list
    data['35d_custom_10'] = data['35d_custom_smoothing']*10
    data['smoothing_factor2'] = smoothing2
    data[str(periods2)+"average"] = data['35d_custom_10'].rolling(window=periods2).mean()
    data = data.dropna()
    smoothing_factor_list2 = [data.iloc[0][str(periods2)+"average"]]
    i=1
    j=0
    while i < len(data[str(periods2)+"average"]):
        smoothing_factor = (data.iloc[i]['35d_custom_10'] - smoothing_factor_list2[j])*data.iloc[i]['smoothing_factor2'] + smoothing_factor_list2[j]
        smoothing_factor_list2.append(smoothing_factor)
        j+=1
        i+=1
    data[str(periods2)+'d_custom_smoothing'] = smoothing_factor_list2
    data[str(periods2)+"_return"] = data['adjClose']/data['adjClose'].shift(periods2)-1
#         data['signal'] = np.where(data[str(periods2)+'d_custom_smoothing'] > data[str(periods2)+'d_custom_smoothing'].shift(1), 1.0, 0.0)
#         data = data.rename(columns={'signal':'price_mo'})
    return data

def get_ichimoku_cloud(dataframe, period1=4, period2=8, period3=15):

    #TODO generate signal, ichimoku works better in current market regime with shorter periods, being able to respond faster to events than a traditional version
    # The conversion crossing the base would be the signal

    data = dataframe.copy()
    data['conversion_line'] = data[['adjClose']].rolling(window=period1).mean()
    data['base_line'] = data[['adjClose']].rolling(window=period2).mean()
    data['senkou_spanA_line'] = (data['conversion_line']+data['base_line'])/2
    data['senkou_spanB_line'] = data[['adjClose']].rolling(window=period3).mean()
    data['lagging_span'] = data['adjClose'].shift(period2)
    data = data.dropna()

    return data

def accumulation_distribution_line(dataframe):

    ##TODO define periodicity and pass as arguments, use the mean as the signal generator, -1 is buy and and 1 is sell
    ##TODO need to add ability to ignore a -1 during a range of 1s

    data = dataframe.copy()
    data['money_flow_mult'] = round(((data['adjClose'] - data['low']) - (data['high'] - data['adjClose']))/(data['high'] - data['low']),2)
    #data = data.dropna()
    data['money_flow_volume'] = data['money_flow_mult']*data['volume']
    money_flow_multiplier_list = list(data['money_flow_volume'].values)
    adl = [money_flow_multiplier_list[0]]
    i = 1
    while i < len(money_flow_multiplier_list):
        a_d_indicator = adl[i-1]+money_flow_multiplier_list[i]
        adl.append(a_d_indicator)
        i+=1
    data['adl'] = adl
    data['adl_change'] = data['adl']/data['adl'].shift(1)-1
    negative_change_count = [0]*9
    i = 0
    counter = 0
    while i < len(data)-9:
        j=0
        while j < 9:
            if data.iloc[j+i]['adl_change'] <0:
                counter+=1
            if j %19 == 0:
                negative_change_count.append(counter)
                counter = 0
            j+=1
        i+=1
    data['negative_change_counter'] = negative_change_count
    data['9_day_return'] = data['adjClose']/data['adjClose'].shift(9)-1
    data['adl_signal'] = rolling_spearman(data['adl'], data['9_day_return'], 9)

    return data

def rsi(dataframe, periods=14):
    data = dataframe.copy()
    data['gains'] = np.where(data['changeOverTime']>0, data['changeOverTime'], 0)
    data['losses'] = np.where(data['changeOverTime']<0, np.absolute(data['changeOverTime']), 0)
    data['average_gain'] = data['gains'].rolling(window=periods).mean()
    data['average_loss'] = data['losses'].rolling(window=periods).mean()
    data['rs'] = data['average_gain']/data['average_loss']
    data['rsi'] = (100 - 100/(1+data['rs']))
    data['rsi_signal'] = data['rsi'].rank(ascending=True, pct=True)

    return data

def get_all_indicators(symbol):
    dataframe = get_dataframe(symbol)
    bb = bollinger_bands(dataframe)
    DEMA = dema(dataframe)
    ADL = accumulation_distribution_line(dataframe)
    RSI = rsi(dataframe)

    dataframe['bollinger_signal'] = bb['bollinger_signal']
    dataframe['dema_signal'] = DEMA['dema_signal']
    dataframe['adl_signal'] = ADL['adl_signal']
    dataframe['rsi_signal'] = RSI['rsi_signal']
    return dataframe

In [32]:
## Steps to take :
## Get updated_df
## Run TI statistics on new dataframe 
## Export new, updated dataframe for use in new models. 

In [33]:
## Constants 

## Set start date variable - api dataframes will be created starting from this date

start_date = '2022-01-01'   ## Use existing dataframes, and add new data to them 
end_date = '2022-01-31'     ## Use end of January - will attempt to predict up to first 10 days 
                            ## of trading for February
    
## Import machine learning list 
path = Path('../Resources/06_01_ML_symbol_success_list.pkl')
machine_learning_dict = load_obj(path)
len(machine_learning_dict)

770

In [34]:
## Test function 
symbol = 'GME'
df = get_all_indicators(symbol)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1511 entries, 2016-02-02 to 2022-01-31
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   open                   1511 non-null   float64
 1   high                   1511 non-null   float64
 2   low                    1511 non-null   float64
 3   close                  1511 non-null   float64
 4   adjClose               1511 non-null   float64
 5   volume                 1511 non-null   float64
 6   unadjustedVolume       1511 non-null   float64
 7   change                 1511 non-null   float64
 8   changePercent          1511 non-null   float64
 9   vwap                   1511 non-null   float64
 10  changeOverTime         1511 non-null   float64
 11  ShortVolumeNSDQ        1511 non-null   float64
 12  ShortExemptVolumeNSDQ  1511 non-null   float64
 13  TotalVolumeNSDQ        1511 non-null   float64
 14  ShortVolumeNYSE        1511 non-null  

In [39]:
df = df.dropna()

In [42]:
df.head(10)

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2016-03-02,30.879999,31.680000,29.160000,31.290001,24.148674,4113100.0,4113100.0,0.41,1.328,30.71000,...,683292.0,2450.0,1313663.0,312907.0,0.0,686106.0,0.146881,0.874917,-0.283333,0.953939
2016-03-03,31.360001,32.950001,31.280001,32.709999,25.244589,3549000.0,3549000.0,1.35,4.305,32.31333,...,638887.0,5799.0,1061679.0,96631.0,0.0,148243.0,0.116700,0.894110,-0.300000,0.975968
2016-03-04,32.270000,32.560001,31.190001,31.230000,24.378124,2530400.0,2530400.0,-1.04,-3.223,31.66000,...,296651.0,800.0,856338.0,30801.0,0.0,109446.0,0.178404,0.876241,0.216667,0.890521
2016-03-07,31.030001,31.700001,30.379999,30.740000,23.995630,2830300.0,2830300.0,-0.29,-0.935,30.94000,...,149749.0,600.0,572103.0,11191.0,0.0,126234.0,0.191147,0.825281,0.500000,0.826435
2016-03-08,30.889999,30.990000,29.809999,30.120001,23.511656,2328500.0,2328500.0,-0.77,-2.493,30.30667,...,164477.0,0.0,477542.0,16400.0,0.0,86438.0,0.267606,0.673726,0.650000,0.684913
2016-03-09,30.379999,31.570000,30.160000,31.290001,24.424961,2594500.0,2594500.0,0.91,2.995,31.00667,...,175497.0,0.0,554868.0,11523.0,0.0,101290.0,0.181757,0.643944,0.700000,0.724299
2016-03-10,31.330000,31.330000,29.940001,30.440001,23.761448,2049300.0,2049300.0,-0.89,-2.841,30.57000,...,277574.0,580.0,446432.0,56527.0,0.0,108167.0,0.195171,0.479153,0.766667,0.746328
2016-03-11,30.540001,31.090000,30.440001,30.750000,24.003435,1619300.0,1619300.0,0.21,0.688,30.76000,...,119463.0,35.0,287684.0,42946.0,0.0,155434.0,0.188464,0.400397,0.800000,0.705607
2016-03-14,30.760000,32.029999,30.719999,31.660000,24.713781,1730400.0,1730400.0,0.90,2.926,31.47000,...,367610.0,41.0,515468.0,61074.0,0.0,90117.0,0.169014,0.449371,0.766667,0.798398


In [44]:
df.tail(23)  ## Loses some of the earlist data when running TI, which is to be expected. But the latest data is most important, and it's all there

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-29,147.85,155.4900,142.1400,153.93,153.93,2029352.0,2029352.0,6.08,4.112,150.52000,...,280068.0,2383.0,627874.0,112580.0,798.0,295581.0,0.923541,0.929186,0.566667,0.427904
2021-12-30,151.00,160.0000,150.0000,155.33,155.33,1556076.0,1556076.0,4.33,2.868,155.11000,...,172058.0,329.0,463841.0,41010.0,0.0,161071.0,0.830315,0.943084,0.533333,0.598798
2021-12-31,153.62,156.6900,148.1001,148.39,148.39,1231436.0,1231436.0,-5.23,-3.405,151.06003,...,137532.0,3387.0,447431.0,28679.0,862.0,128247.0,0.892019,0.941760,0.450000,0.548064
2022-01-03,149.23,159.3800,149.2100,152.84,152.84,1391973.0,1391973.0,3.61,2.419,153.81000,...,158387.0,1740.0,414886.0,16842.0,108.0,90980.0,0.780013,0.948379,0.400000,0.849800
2022-01-04,152.17,152.9900,143.0500,148.91,148.91,1827668.0,1827668.0,-3.26,-2.142,148.31667,...,317436.0,463.0,546744.0,63862.0,48.0,129093.0,0.753186,0.947717,0.450000,0.620828
2022-01-05,148.39,148.8500,129.1501,129.37,129.37,3341849.0,3341849.0,-19.02,-12.818,135.79003,...,551519.0,4959.0,925958.0,141401.0,4796.0,306137.0,0.945003,0.788882,0.683333,0.311749
2022-01-06,132.84,137.7000,121.1400,131.03,131.03,3435686.0,3435686.0,-1.81,-1.363,129.95667,...,450886.0,11530.0,861811.0,146713.0,9208.0,355103.0,0.931590,0.094639,0.800000,0.383845
2022-01-07,159.77,160.3100,132.5000,140.62,140.62,12125225.0,12125225.0,-19.15,-11.986,144.47667,...,1924724.0,12494.0,3779718.0,566527.0,1213.0,1199201.0,0.714956,0.260093,0.716667,0.095461
2022-01-10,134.99,135.6900,120.0000,131.15,131.15,5505944.0,5505944.0,-3.84,-2.845,128.94667,...,874373.0,15128.0,1646338.0,201061.0,20288.0,481043.0,0.869886,0.099934,0.716667,0.052737


In [ ]:
# symbol_success_list = []
# for symbol in machine_learning_dict:
#     temp_dict = {}
#     try:
#         df = get_all_indicators(symbol)
#     except: continue 
        
#     df = df.dropna()
    
#     if len(df) > 1400:
#         temp_dict[symbol] = df
#         #data_dict[symbol] = df 
#         symbol_success_list.append(symbol)
        
#         ## Export Temp dict 
#         export_path = Path('../FilesExport_Updated_DFs_01_31/'+symbol+'_ti_df_no_ftd.pkl')
#         save_obj(temp_dict,export_path)
        
        
# len(symbol_success_list) 

In [ ]:
# ## Export symbol_success_list as ML_symbol_success_list.pkl
# export_path = Path('../Resources/Updated_01_DF_success_list.pkl')
# save_obj(symbol_success_list,export_path)